In [ ]:
from trading_ig import IGService,IGStreamService
from trading_ig.lightstreamer import LSClient, Subscription
import pandas as pd
import numpy as np
import configparser
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mplfinance.original_flavor import candlestick_ohlc
import mplfinance as mpf
import traceback
import sys
import atexit
from collections import deque


from lightstreamer.client import (
    LightstreamerClient,
    Subscription,
    ConsoleLoggerProvider,
    ConsoleLogLevel,
    SubscriptionListener,
    ItemUpdate
)



Ställer in vart loggade meddelandet hamnar

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Ta bort alla befintliga loggningshanterare
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging_path = r'C:\Dev\AlgoTrading\log.txt'
logging.basicConfig(filename=logging_path,
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')



In [ ]:
class Streaming_client():

    def __init__(self, config_file_path):
        self.config_file_path = config_file_path
        self.api_key = self.read_config('Credentials', 'apikey')
        self.username = self.read_config('Credentials', 'username')
        self.password = self.read_config('Credentials', 'password')
        self.acc_type = 'DEMO'
        self.ig_service = IGService(self.username, self.password, self.api_key, self.acc_type)
        self.ig_service.create_session()
        self.ig_stream_service = IGStreamService(self.ig_service)
        self.ig_stream_service.create_session()
        self.market_listener = self.MarketListener()
        self.tick_data_listener = self.TickDataListener()
        self.candle_data_listener = self.CandleDataListener()

    def read_config(self, section, key):
        config = configparser.ConfigParser()
        try:
            config.read(self.config_file_path)
            return config.get(section, key)
        except Exception as e:
            logger.error(f"Ett fel uppstod när konfigurationsfilen skulle läsas: {e}")
            return None


    def get_market_updates(self):
        return self.market_listener.get_updates()
    
    def get_tick_data_updates(self):
        return self.tick_data_listener.get_updates()
    
    def get_candle_data_updates(self):
        return self.candle_data_listener.get_updates()
    
    def start_market_streaming(self, epics):
        # Create a new MARKET Subscription
        market_subscription = Subscription(
            mode="MERGE",
            items=[f"MARKET:{epic}" for epic in epics],
            fields=["UPDATE_TIME", "BID", "OFFER", "CHANGE", "MARKET_STATE", "CHANGE_PCT", "HIGH", "LOW"]
        )
        market_subscription.addListener(self.market_listener)
        self.ig_stream_service.subscribe(market_subscription)

    def subscribe_to_tick_data(self, epic):
        subscription = Subscription(
            mode="DISTINCT",
            items=[f"CHART:{epic}:TICK"],
            fields=[
                "BID", "OFR", "LTP", "LTV", "TTV", "UTM", "DAY_OPEN_MID",
                "DAY_NET_CHG_MID", "DAY_PERC_CHG_MID", "DAY_HIGH", "DAY_LOW"
            ]
        )
        subscription.addListener(self.tick_data_listener)
        self.ig_stream_service.subscribe(subscription)

    def subscribe_to_candle_data(self, epic, scale):
        subscription = Subscription(
            mode="MERGE",
            items=[f"CHART:{epic}:{scale}"],
            fields=[
                "LTV", "TTV", "UTM", "DAY_OPEN_MID", "DAY_NET_CHG_MID", 
                "DAY_PERC_CHG_MID", "DAY_HIGH", "DAY_LOW", "OFR_OPEN", "OFR_HIGH",
                "OFR_LOW", "OFR_CLOSE", "BID_OPEN", "BID_HIGH", "BID_LOW",
                "BID_CLOSE", "LTP_OPEN", "LTP_HIGH", "LTP_LOW", "LTP_CLOSE",
                "CONS_END", "CONS_TICK_COUNT"
            ]
        )
        subscription.addListener(self.candle_data_listener)
        self.ig_stream_service.subscribe(subscription)
    
    def disconnect_stream(self):
        """Stäng av strömningssessionen."""
        try:
            self.ig_stream_service.disconnect()
            print("Strömningssessionen har avslutats.")
        except Exception as e:
            print(f"Ett fel uppstod när strömningen skulle avslutas: {e}")

    class MarketListener(SubscriptionListener):
        def __init__(self):
            self.updates = deque(maxlen=10)  # Håller de senaste 10 uppdateringarna
        
        def onItemUpdate(self, update: ItemUpdate):
            data = (
                f"{update.getValue('UPDATE_TIME')} {update.getItemName()} "
                f"Bid: {update.getValue('BID')}, "
                f"Offer: {update.getValue('OFFER')}, "
                f"Price change: {update.getValue('CHANGE')}, "
                f"State: {update.getValue('MARKET_STATE')}, "
                f"Change: {update.getValue('CHANGE_PCT')}%, "
                f"High: {update.getValue('HIGH')}, "
                f"Low: {update.getValue('LOW')}"
            )
            self.updates.append(data)
        
        def get_updates(self):
            return self.updates
        

    class TickDataListener(SubscriptionListener):
        def __init__(self):
            self.updates = deque(maxlen=10)  # Håller de senaste 10 uppdateringarna
            
        def onItemUpdate(self, update: ItemUpdate):
            data = (f"Time: {update.getValue('UTM')}, "
                f"Bid: {update.getValue('BID')}, "
                f"Offer: {update.getValue('OFR')}, "
                f"Last Traded Price: {update.getValue('LTP')}, "
                f"Last Traded Volume: {update.getValue('LTV')}, "
                f"Incremental Trading Volume: {update.getValue('TTV')}, "
                f"Day Open Mid: {update.getValue('DAY_OPEN_MID')}, "
                f"Change from Open: {update.getValue('DAY_NET_CHG_MID')}, "
                f"Daily Percentage Change: {update.getValue('DAY_PERC_CHG_MID')}, "
                f"Daily High: {update.getValue('DAY_HIGH')}, "
                f"Daily Low: {update.getValue('DAY_LOW')}"
            )
            #print(f"data received: {data}")
            self.updates.append(data)
        
        def get_updates(self):
            return self.updates
        

    class CandleDataListener(SubscriptionListener):
        def __init__(self):
            self.updates = deque(maxlen=10)  # Håller de senaste 10 uppdateringarna

        def onItemUpdate(self, update: ItemUpdate):
            data = {
                "LTV": update.getValue('LTV'),
                "TTV": update.getValue('TTV'),
                "UTM": update.getValue('UTM'),
                "DAY_OPEN_MID": update.getValue('DAY_OPEN_MID'),
                "DAY_NET_CHG_MID": update.getValue('DAY_NET_CHG_MID'),
                "DAY_PERC_CHG_MID": update.getValue('DAY_PERC_CHG_MID'),
                "DAY_HIGH": update.getValue('DAY_HIGH'),
                "DAY_LOW": update.getValue('DAY_LOW'),
                "OFR_OPEN": update.getValue('OFR_OPEN'),
                "OFR_HIGH": update.getValue('OFR_HIGH'),
                "OFR_LOW": update.getValue('OFR_LOW'),
                "OFR_CLOSE": update.getValue('OFR_CLOSE'),
                "BID_OPEN": update.getValue('BID_OPEN'),
                "BID_HIGH": update.getValue('BID_HIGH'),
                "BID_LOW": update.getValue('BID_LOW'),
                "BID_CLOSE": update.getValue('BID_CLOSE'),
                "LTP_OPEN": update.getValue('LTP_OPEN'),
                "LTP_HIGH": update.getValue('LTP_HIGH'),
                "LTP_LOW": update.getValue('LTP_LOW'),
                "LTP_CLOSE": update.getValue('LTP_CLOSE'),
                "CONS_END": update.getValue('CONS_END'),
                "CONS_TICK_COUNT": update.getValue('CONS_TICK_COUNT')
            }
            self.updates.append(data)

        def get_updates(self):
            return self.updates


    


In [ ]:
class CandlestickVisualizer:
    def __init__(self):
        pass

    def plot_candlestick(self, data):
        # Konvertera data till DataFrame
        df = pd.DataFrame(list(data))

        df['UTM'] = df['UTM'].astype('int64')
        df['Date'] = pd.to_datetime(df['UTM'], unit='ms')
        df.set_index('Date', inplace=True)

        # Konvertera kolumner till float-typ
        columns_to_convert = ['OFR_OPEN', 'OFR_HIGH', 'OFR_LOW', 'OFR_CLOSE', 
                            'BID_OPEN', 'BID_HIGH', 'BID_LOW', 'BID_CLOSE',
                            'LTP_OPEN', 'LTP_HIGH', 'LTP_LOW', 'LTP_CLOSE', 'LTV']
        
        for column in columns_to_convert:
            df[column] = df[column].replace('', np.nan).astype(float)
            
        # Namnge kolumnerna enligt vad mplfinance förväntar sig
        df = df.rename(columns={
            'OFR_OPEN': 'Open',
            'OFR_HIGH': 'High',
            'OFR_LOW': 'Low',
            'OFR_CLOSE': 'Close',
            'LTV': 'Volume'
        })

        # Skapa ljusstakdiagrammet
        mpf.plot(df, type='candle', style='charles',
                 title='Candlestick Chart',
                 ylabel='Price',
                 volume=True,
                 show_nontrading=True)


#### Börja strömma
1. Skapar en instans av klassen <br> 
2. Väljer vilket finansiellt instrumment vi vill strömma mot <br>
3. Initierar strömming till log filen på avsatt ställe


In [ ]:
config_file_path = r"C:\Dev\config\aktiebot.cfg"
klient = Streaming_client(config_file_path)
epics = ['CS.D.USDSEK.CFD.IP']
klient.subscribe_to_candle_data(epics[0], '1MINUTE')

In [ ]:
# Skapa en instans av visualiseraren
visualizer = CandlestickVisualizer()

# Hämta din ljusstakedata, antar att du har en instans av `Streaming_client` som heter `client`
data = klient.get_candle_data_updates()

# Plotta ljusstakdiagrammet
visualizer.plot_candlestick(data)


In [ ]:
print(data)

In [ ]:
""" latest_market_updates = klient.get_market_updates()
print(latest_market_updates) """

""" latest_tick_data_updates = klient.get_tick_data_updates()
print(latest_tick_data_updates) """
""" 
latest_candle_data_updates = klient.get_candle_data_updates()
print(latest_candle_data_updates) """




In [ ]:
#klient.disconnect_stream()